# Data processing

The water monitoring project at the Montreux Jazz has been going on since 2016. The data has been collected and treated by a variety of people since then. 

__Objective:__ Standardize the nomenclature from the different sampling years. Provide a model for storing and collecting data in the future.

__Purpose:__ Define the probability that a survey will exceed a threshold value within the period of the year defined by the survey results.

## Definitions

* colony: a circular discoloration of the media within a defined size and color range
* colony-count: the number of discolorations of the same hue for a media type
* media/medium: the environment that the water samples are placed in
* color: the observed color of the colony
* label: the assumed category of the color:
  * Bioindicator
  * Coliform
  * Other
* coef:  the correction factor applied, to allow reporting of colony counts per 100ml of the original water sample.
 
The purpose of the sampling is to identify colonies that appear in the media and classify them as one of the possible labels. The label of interest is _Bioindicators_, this represents the bacteria that are issue from the organism of interest. The organism in this case is people, the _Bioindicator_ is issue from fecal contaminants.

## Methods

The process requires collaborating with the data-manager(s) from the different project years and ensuring that the data from each year can be combined and interpreted together. The data for this collaboration is stored in the _componentdata_ folder.

The relationship of previous label <---> new label is stored in a dictionary or an array for the different possibilities of medium, color, label and coefficient. The new labels are applied to a data-frame.

The finsihed data (the result of the collaboration) is stored in the _end_ folder

## Sample data

The sample data is an example of the desired output per year. This includes the following parameters:

1. colony-count
2. label
3. location
4. coeficient*count
5. week number
6. day of year
7. is-jazz: boolean
8. rain fall in millimeters

In [1]:
import pandas as pd
import datetime as dt
import numpy as np

project = "Hackuarium do it together water quality sampling"
site_markers = {"SVT":"o", "VNX":"D", "MRD":"X"}
species_colors = { "Bioindicator":"dodgerblue", "Coliform":"magenta"}
marker_colors = {"SVT":"black", "VNX":"green", "MRD":"goldenrod"}
sites = ["SVT", "VNX", "MRD"]

## Survey data

The format of the survey data prior to processing. The result of the collaboration.

In [2]:
stddf = pd.read_csv("data/end/survey_data_2020_2023.csv")
stddf['date'] = pd.to_datetime(stddf["date"])
# stddf["year"] = stddf["year"].astype("str")
# stddf["year"] = stddf["date"].dt.year
# stddf.drop_duplicates(["date", "sample", "label", "medium", "year"], inplace=True)
# stddf["year"] = stddf["year"].astype("str")
stddf.head()

,date,sample,temperature,media,color,count,image (48h),coef,date_sample,year,location,doy,week,event,label,medium,before event,after event
0,2023-06-12,VNX1,16.1,ECC-A,Dark Blue,0,20230614_205159.jpg,100.0,"('12/6/2023', 'VNX1')",2023,VNX,163,24,False,Bioindicator,ECC-A,True,False
1,2023-06-12,VNX1,16.1,ECC-A,Turquoise,0,20230614_205159.jpg,100.0,"('12/6/2023', 'VNX1')",2023,VNX,163,24,False,Other,ECC-A,True,False
2,2023-06-12,VNX1,16.1,ECC-A,Pink,11,20230614_205159.jpg,100.0,"('12/6/2023', 'VNX1')",2023,VNX,163,24,False,Coliform,ECC-A,True,False
3,2023-06-12,VNX2,16.1,ECC-A,Dark Blue,1,20230614_205229.jpg,100.0,"('12/6/2023', 'VNX2')",2023,VNX,163,24,False,Bioindicator,ECC-A,True,False
4,2023-06-12,VNX2,16.1,ECC-A,Turquoise,1,20230614_205229.jpg,100.0,"('12/6/2023', 'VNX2')",2023,VNX,163,24,False,Other,ECC-A,True,False


In [3]:
df = pd.read_csv("data/componentdata/survey_data_2020.csv")
df["date"] = pd.to_datetime(df["date"])

In [4]:
new_df16 = pd.read_csv("data/componentdata/2016_Data.csv")

colors_2016 = [
    'P1_24h_big_blue', 'P1_24h_med_blue',
       'P1_24h_other', 'P1_24h_pink', 'P1_24h_turq', 'P1_qty_sample',
       'P2_24h_big_blue', 'P2_24h_med_blue', 'P2_24h_other', 'P2_24h_pink',
       'P2_24h_turq', 'P3_24h_big_blue', 'P3_24h_med_blue', 'P3_24h_other',
       'P3_24h_pink', 'P3_24h_turq']
new_df16.columns

Index(['Unnamed: 0', 'Date', 'Location', 'P1_24h_big_blue', 'P1_24h_med_blue',
       'P1_24h_other', 'P1_24h_pink', 'P1_24h_turq', 'P1_qty_sample',
       'P2_24h_big_blue', 'P2_24h_med_blue', 'P2_24h_other', 'P2_24h_pink',
       'P2_24h_turq', 'P3_24h_big_blue', 'P3_24h_med_blue', 'P3_24h_other',
       'P3_24h_pink', 'P3_24h_turq'],
      dtype='object')

In [5]:
new_df17 = pd.read_csv("data/componentdata/2017_Data.csv")
new_df16.columns

Index(['Unnamed: 0', 'Date', 'Location', 'P1_24h_big_blue', 'P1_24h_med_blue',
       'P1_24h_other', 'P1_24h_pink', 'P1_24h_turq', 'P1_qty_sample',
       'P2_24h_big_blue', 'P2_24h_med_blue', 'P2_24h_other', 'P2_24h_pink',
       'P2_24h_turq', 'P3_24h_big_blue', 'P3_24h_med_blue', 'P3_24h_other',
       'P3_24h_pink', 'P3_24h_turq'],
      dtype='object')

### Applying labels

The colors that were used for the observations can be placed into three broad categories. 

1. Bioindicator
2. Coliforms
3. Other

The microbiologist determines the correct label for the recorded color based on the specifics of the media/medium used to grow the culture.

The colors appropriate to each label are stored in an array. The color for each record is tested for membership in one of the arrays. If it is in one of the arrays, the name of that array is returned. If the color is not in any array the original value is returned. The result is added to the data-frame.

```python
bioindicators = ["Dark Blue", "Blue", "Turquoise fast", "metallic_green", "green_met"]
coliforms = ["Pink", "pink"]
other = ["Turquoise", "Turquoise slow", "other"]

def translate_colors(x, bioindicators, coliforms, other):
    if x in bioindicators:
        return "Bioindicator"
    elif x in coliforms:
        return "Coliform"
    elif x in other:
        return "Other"
    else:
        return x

stddf ["label"] = stddf .color.apply(lambda x: translate_colors(x, bioindicators, coliforms, other))
```

We do the same for the media/medium except we use a dictionary to store that information

```python
media_names = {
    "ECC-A Card":"ECC-A",
    "new ECCA":"ECC-A",
    "E-coli side": "E coli",
    "ECC-side":"ECC",
    "selective":"Levine",
    "media":"EasyGel",
    "plus uv":"EasyGelPlus",
    "UVplus":"EasyGelPlus",
    "non-restrictive":"LB"
}

def translate_media(x, media_names):
    if x in media_names.keys():
        return media_names[x]
    else:
        return x


stddf ["medium"] = stddf .media.apply(lambda x: translate_media(x, media_names))

```



### Labeling the date range of interest

Voici les dates de Jazz pour toutes les années de prélèvement :

* 2016:  2016-07-01 - 2016-07-16
* 2017: 2017-06-30 - 2017-06-15
* 2020: 2020-07-03 - 2020-07-18
* 2022: 2022-07-01 - 2022-07-16
* 2023: 2023-06-30 - 2023-07-15

__before event:__ samples before the begining of the event of interest

__after event:__ samples after the end of the event

In [6]:
stddf.head()

,date,sample,temperature,media,color,count,image (48h),coef,date_sample,year,location,doy,week,event,label,medium,before event,after event
0,2023-06-12,VNX1,16.1,ECC-A,Dark Blue,0,20230614_205159.jpg,100.0,"('12/6/2023', 'VNX1')",2023,VNX,163,24,False,Bioindicator,ECC-A,True,False
1,2023-06-12,VNX1,16.1,ECC-A,Turquoise,0,20230614_205159.jpg,100.0,"('12/6/2023', 'VNX1')",2023,VNX,163,24,False,Other,ECC-A,True,False
2,2023-06-12,VNX1,16.1,ECC-A,Pink,11,20230614_205159.jpg,100.0,"('12/6/2023', 'VNX1')",2023,VNX,163,24,False,Coliform,ECC-A,True,False
3,2023-06-12,VNX2,16.1,ECC-A,Dark Blue,1,20230614_205229.jpg,100.0,"('12/6/2023', 'VNX2')",2023,VNX,163,24,False,Bioindicator,ECC-A,True,False
4,2023-06-12,VNX2,16.1,ECC-A,Turquoise,1,20230614_205229.jpg,100.0,"('12/6/2023', 'VNX2')",2023,VNX,163,24,False,Other,ECC-A,True,False


In [25]:
# mask the date ranges
import datetime as dt

def make_date_object(x):
    return dt.datetime.strptime(x, "%Y-%m-%d")

# # stddf["before event"] = False
# # stddf["after event"] = False
df["event"] = False
event_mask = (df['date'] >= pd.Timestamp("2020-07-01")) & (df['date'] > pd.Timestamp("2020-07-16"))
df.loc[event_mask, "event"] = True

df["before event"] = False
df["after event"] = False

df.loc[ (df['date'] < pd.Timestamp("2020-07-01")), "before event"] = True
df.loc[ (df['date'] > pd.Timestamp("2020-07-16")), "after event"] = True

# stddf["year"] = stddf["year"].astype("str")
# stddf.rename(columns={"isjazz": "event"}, inplace=True)

# stddf.loc[(stddf["year"] == "2022") & (stddf['date'] < pd.Timestamp("2022-07-01")), "before event"] = True
# stddf.loc[(stddf["year"] == "2023") & (stddf['date'] < pd.Timestamp("2023-06-30")), "before event"] = True
# stddf.loc[(stddf["year"] == "2022") & (stddf['date'] > pd.Timestamp("2022-07-16")), "after event"] = True
# stddf.loc[(stddf["year"] == "2023") & (stddf['date'] > pd.Timestamp("2023-07-15")), "after event"] = True
stddf.head()

,date,sample,temperature,media,color,count,image (48h),coef,date_sample,year,location,doy,week,event,label,medium,before event,after event
0,2023-06-12,VNX1,16.1,ECC-A,Dark Blue,0,20230614_205159.jpg,100.0,"('12/6/2023', 'VNX1')",2023,VNX,163,24,False,Bioindicator,ECC-A,True,False
1,2023-06-12,VNX1,16.1,ECC-A,Turquoise,0,20230614_205159.jpg,100.0,"('12/6/2023', 'VNX1')",2023,VNX,163,24,False,Other,ECC-A,True,False
2,2023-06-12,VNX1,16.1,ECC-A,Pink,11,20230614_205159.jpg,100.0,"('12/6/2023', 'VNX1')",2023,VNX,163,24,False,Coliform,ECC-A,True,False
3,2023-06-12,VNX2,16.1,ECC-A,Dark Blue,1,20230614_205229.jpg,100.0,"('12/6/2023', 'VNX2')",2023,VNX,163,24,False,Bioindicator,ECC-A,True,False
4,2023-06-12,VNX2,16.1,ECC-A,Turquoise,1,20230614_205229.jpg,100.0,"('12/6/2023', 'VNX2')",2023,VNX,163,24,False,Other,ECC-A,True,False


In [26]:
# translate colors
def translate_colors(x, bioindicators, coliforms, other):
    if x in bioindicators:
        return "Bioindicator"
    elif x in coliforms:
        return "Coliform"
    elif x in other:
        return "Other"
    else:
        return x

bioindicators = ["Dark Blue", "Blue", "Turquoise fast", "metallic_green", "green_met", "fluo_halo", "big_blue"]
coliforms = ["Pink", "pink", "purple", "med_blue"]
other = ["Turquoise", "Turquoise slow", "other", "mauve", "fluo_other", "green"]

df["label"] = df.color.apply(lambda x: translate_colors(x, bioindicators, coliforms, other))

def translate_media(x, media_names):
    if x in media_names.keys():
        return media_names[x]
    else:
        return x

media_names =  {
"ECC-A Card":"ECC-A",
"new ECCA":"ECC-A",
"E-coli side": "E coli",
"Double side E coli": "E coli",
"ECC-side":"ECC",
"Double side ECC":"ECC",
"selective":"Levine",
"media":"EasyGel",
"plus uv":"EasyGelPlus",
"UVplus":"EasyGelPlus",
"non-restrictive":"LB",
"levine": "Levine",
"easy_gel":"EasyGel",
"unil_kitchen":"LB",
"micrology_card": "ECC"
}

df["medium"] = df.media.apply(lambda x: translate_media(x, media_names))

# Rain fall

In [27]:
sample_data = pd.read_csv("data/end/rain_data_2016.csv")
sample_data.head()

,date,mm
0,2016-06-21,4.0
1,2016-06-22,0.6
2,2016-06-23,0.9
3,2016-06-24,13.1
4,2016-06-25,9.8


In [28]:
stddf ["medium"].unique()

array(['ECC-A', 'ECC', 'E coli', 'LB', 'Levine'], dtype=object)

In [29]:
df["medium"].unique()

array(['LB', 'Levine'], dtype=object)

In [30]:
df["label"].unique()

array(['Bioindicator', 'Coliform', 'Other'], dtype=object)

In [31]:
df.coef.unique()

array([1.])

In [32]:
df.year.unique()

array([2020])

In [33]:
df["date"] = pd.to_datetime(df["date"])
df["week"] = df["date"].dt.isocalendar().week

In [34]:
df.week.unique()

<IntegerArray>
[24, 25, 26, 27, 28, 29, 30, 32]
Length: 8, dtype: UInt32

In [35]:
stddf  = stddf [stddf .location.isin(sites)]
stddf ["count"] = stddf ["count"].astype('int', errors='ignore')
dropthese = stddf[stddf ["count"] == 'nd']
dropthese

,date,sample,temperature,media,color,count,image (48h),coef,date_sample,year,location,doy,week,event,label,medium,before event,after event


In [36]:
# stddf.to_csv("data/end/survey_data_2020_2023.csv", index=False)

In [37]:
stddf.head()

,date,sample,temperature,media,color,count,image (48h),coef,date_sample,year,location,doy,week,event,label,medium,before event,after event
0,2023-06-12,VNX1,16.1,ECC-A,Dark Blue,0,20230614_205159.jpg,100.0,"('12/6/2023', 'VNX1')",2023,VNX,163,24,False,Bioindicator,ECC-A,True,False
1,2023-06-12,VNX1,16.1,ECC-A,Turquoise,0,20230614_205159.jpg,100.0,"('12/6/2023', 'VNX1')",2023,VNX,163,24,False,Other,ECC-A,True,False
2,2023-06-12,VNX1,16.1,ECC-A,Pink,11,20230614_205159.jpg,100.0,"('12/6/2023', 'VNX1')",2023,VNX,163,24,False,Coliform,ECC-A,True,False
3,2023-06-12,VNX2,16.1,ECC-A,Dark Blue,1,20230614_205229.jpg,100.0,"('12/6/2023', 'VNX2')",2023,VNX,163,24,False,Bioindicator,ECC-A,True,False
4,2023-06-12,VNX2,16.1,ECC-A,Turquoise,1,20230614_205229.jpg,100.0,"('12/6/2023', 'VNX2')",2023,VNX,163,24,False,Other,ECC-A,True,False


In [45]:
merge_columns = [
    "date",
    "location",
    "sample",
    "date_sample",
    "event",
    "before event",
    "after event",
    "medium",
    "label",
    "count",
    "coef",
    "week",
    "doy",
    "year",
    "color",
    "image",
    
]
df["image"] = df["image(48h)"]
stddf["image"] =  stddf["image (48h)"]
df["date_sample"] = list(zip(df['date'].dt.date, df['sample']))


In [46]:
stddf["coef"] = 100
df["coef"] = 100

In [47]:
df[merge_columns].head()

,date,location,sample,date_sample,event,before event,after event,medium,label,count,coef,week,doy,year,color,image
0,2020-06-11,VNX,VNX1,"(2020-06-11, VNX1)",False,True,False,LB,Bioindicator,0.0,100,24,163,2020,green_met,IMG_9310.JPG
1,2020-06-11,VNX,VNX1,"(2020-06-11, VNX1)",False,True,False,Levine,Bioindicator,0.0,100,24,163,2020,green_met,IMG_9311.JPG
2,2020-06-11,VNX,VNX2,"(2020-06-11, VNX2)",False,True,False,Levine,Bioindicator,0.0,100,24,163,2020,green_met,IMG_9312.JPG
3,2020-06-11,VNX,VNX3,"(2020-06-11, VNX3)",False,True,False,Levine,Bioindicator,0.0,100,24,163,2020,green_met,IMG_9313.JPG
4,2020-06-11,SVT,SVT1,"(2020-06-11, SVT1)",False,True,False,LB,Bioindicator,0.0,100,24,163,2020,green_met,IMG_9314.JPG


In [48]:
stddf[merge_columns].head()

,date,location,sample,date_sample,event,before event,after event,medium,label,count,coef,week,doy,year,color,image
0,2023-06-12,VNX,VNX1,"('12/6/2023', 'VNX1')",False,True,False,ECC-A,Bioindicator,0,100,24,163,2023,Dark Blue,20230614_205159.jpg
1,2023-06-12,VNX,VNX1,"('12/6/2023', 'VNX1')",False,True,False,ECC-A,Other,0,100,24,163,2023,Turquoise,20230614_205159.jpg
2,2023-06-12,VNX,VNX1,"('12/6/2023', 'VNX1')",False,True,False,ECC-A,Coliform,11,100,24,163,2023,Pink,20230614_205159.jpg
3,2023-06-12,VNX,VNX2,"('12/6/2023', 'VNX2')",False,True,False,ECC-A,Bioindicator,1,100,24,163,2023,Dark Blue,20230614_205229.jpg
4,2023-06-12,VNX,VNX2,"('12/6/2023', 'VNX2')",False,True,False,ECC-A,Other,1,100,24,163,2023,Turquoise,20230614_205229.jpg


In [52]:
end_df = pd.concat([stddf[merge_columns], df[merge_columns]])
end_df.to_csv("data/end/survey_data_2020_2023.csv", index=False)